#### iris 데이터셋 활용 꽃잎 너비 예측 모델
- 데이터셋 :  iris.csv에서 2개  Feature 사용
- 구현프레임워크 : Pytorch

- [1] 모듈 로딩 및 데이터 준비

In [1]:
# 모듈 로딩 
import torch                                            # 텐서 및 수치 계산 함수 관련 모듈
import torch.nn as nn                                   # 인공신경망 관련 모듈
import torch.nn.functional as F                         # 손실, 거리 등 함수 관련 모듈
import torch.optim as optimizer                         # 최적화 기법 관련 모듈
from torchmetrics.regression import R2Score             # 성능지표 관련 모듈  - 추가 설치
from torchinfo import summary                           # 모델 정보 관련 모듈 - 추가 설치

import pandas as pd                                     # 데이터 파일 분석 관련 모듈 
from sklearn.model_selection import train_test_split 

In [2]:
# 모델의 가중치 및 절편 값 고정 설정
torch.manual_seed(1)

# 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f'DEVICE => {DEVICE}')

DEVICE => cpu


In [3]:
# 데이터 로딩 : CSV => DataFrame
DATA_FILE='../data/iris.csv'

irisDF=pd.read_csv(DATA_FILE, usecols=[0, 1, 2, 3])
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


[2] 모델 준비
- 학습방법 : 지도학습 > 회귀
- 알고리즘 : 선형관계 >> 선형모델 ==> nn.Linear

In [8]:
# 모델 설계
# 입력층에 입력값/피쳐 => sepal.length, sepal.width,	petal.length 3개
# 출력층의 출력값/타겟 => petal.width 1개
# 입력층 : 입력 피쳐3    출력 존재 퍼셉트론 개수 10   AF ReLU
#               ________|
#              | ReLU
#              V
# 은닉층 : 입력 10      출력 존재 퍼셉트론 개수 5     AF ReLU 
#               ________|
#              |ReLU
#              V
# 출력층 : 입력 5       출력 타겟/라벨 개수 1        AF - None
model = nn.Sequential(   nn.Linear(3, 10), 
                            nn.ReLU(),
                            nn.Linear(10, 5), 
                            nn.ReLU(),
                            nn.Linear(5, 1))

In [9]:
# 모델 구조 확인
print(model)
summary(model)

Sequential(
  (0): Linear(in_features=3, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=1, bias=True)
)


Layer (type:depth-idx)                   Param #
Sequential                               --
├─Linear: 1-1                            40
├─ReLU: 1-2                              --
├─Linear: 1-3                            55
├─ReLU: 1-4                              --
├─Linear: 1-5                            6
Total params: 101
Trainable params: 101
Non-trainable params: 0

In [10]:
# 가중치와 절편 확인
for name, param in model.named_parameters():
    print(f'[{name}] {param}\n')

[0.weight] Parameter containing:
tensor([[ 0.0321, -0.2970, -0.5503],
        [-0.1848, -0.0413, -0.1058],
        [ 0.3760, -0.3809,  0.4897],
        [ 0.4869,  0.1013, -0.5422],
        [-0.1322,  0.0329, -0.3658],
        [-0.3760, -0.0571, -0.4696],
        [-0.1574, -0.2888,  0.2002],
        [ 0.4029, -0.4003,  0.0340],
        [ 0.3738, -0.4389,  0.2620],
        [-0.4557, -0.3214,  0.0880]], requires_grad=True)

[0.bias] Parameter containing:
tensor([-0.1182, -0.2484,  0.2063,  0.2994,  0.0768, -0.4408, -0.5349, -0.2755,
        -0.1677,  0.1404], requires_grad=True)

[2.weight] Parameter containing:
tensor([[ 0.0370, -0.1650, -0.0185,  0.2833, -0.0741, -0.0943,  0.1939,  0.2787,
          0.2057,  0.1269],
        [-0.0020, -0.2696, -0.1175,  0.2735,  0.3046,  0.2114, -0.0974, -0.2835,
          0.1481,  0.0513],
        [ 0.2878, -0.0050,  0.2064, -0.2740,  0.1746, -0.2022, -0.1057, -0.1682,
         -0.1472, -0.0402],
        [-0.2001, -0.1701, -0.0972,  0.1446, -0.1636, -0

[3]최적화 인스턴스 준비 

In [11]:
### 모델의 가중치와 절편을 최적화 ==> 인스턴스에 전달
adam_optim=optimizer.Adam(model.parameters(), lr=0.1)

[4]학습 ==> 개발자가 구현

-[4-1] 데이터셋 Tensor화 진행 : 데이터준비 시 진행 하거나 또는 학습 전 진행

In [12]:
# 피쳐와 타겟 분리
featureDF=irisDF[irisDF.columns[:-1]]
targetDF=irisDF[['petal.width']]

print(featureDF.shape, targetDF.shape )

(150, 3) (150, 1)


In [13]:
# Train & Test
X_train, X_test, y_train, y_test = train_test_split(featureDF, 
                                                    targetDF, 
                                                    test_size=0.2,
                                                    random_state=5)
# Train & Valid
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                    y_train, 
                                                    test_size=0.2,
                                                    random_state=5)

print(f'[FEATURE] TRAIN {X_train.shape}, TEST {X_test.shape}, VAL {X_val.shape}')
print(f'[TARGET]  TRAIN {y_train.shape}, TEST {y_test.shape}, VAL {y_val.shape}')


[FEATURE] TRAIN (96, 3), TEST (30, 3), VAL (24, 3)
[TARGET]  TRAIN (96, 1), TEST (30, 1), VAL (24, 1)


- [4-2] 학습진행 
    * 학습횟수 결정 ==> 에포크 설정
    * 배치크기 결정 
    * 배치개수 계산 

In [14]:
import math as m

In [15]:
96/12, m.ceil(91/10)

(8.0, 10)

In [17]:
EPOCH=10000                                  # 처음~끝까지 공부하는 횟수
BATCH_SIZE= 12                              # 1에포크에서 한 번 학습할 분량 크기
BATCH_CNT= X_train.shape[0]//BATCH_SIZE     # 1에포크에서 총 학습 횟수이면서 업데이트 횟수

print(f'EPOCH:{EPOCH}, BATCH_SIZE:{BATCH_SIZE}, BATCH_CNT:{BATCH_CNT}')

EPOCH:10000, BATCH_SIZE:12, BATCH_CNT:8


In [18]:
## 테스트/검증 함수 
#  ==> 가중치, 절편 업데이트 X, 최적화 미진행
#  ==> 현재 가중치와 절편값으로 테스트 진행 
def testing( testDF, targetDF, kind='Val'):
    # Tensor화
    testTS = torch.FloatTensor(testDF.values).to(DEVICE)
    targetTS = torch.FloatTensor(targetDF.values).to(DEVICE)
    
    with torch.no_grad():    # 가중치 및 절편 업데이트 진행 X
        #-(1)학습진행 - forward
        pre_y=model(testTS)
        print(f'{pre_y.shape}')
        
        #-(2)오차계산 - 손실함수
        loss=F.mse_loss(pre_y, targetTS)
        
        #-(3)성능평가 - R2
        r2 =R2Score()(pre_y, targetTS)
        
        #-(4)학습결과 출력 및 저장
        print(f'[{kind}] LOSS : {loss}, R2 : {r2}')
        
    return loss , r2


In [19]:
X_train.shape[0], X_train.shape[0]//BATCH_SIZE

(96, 8)

In [20]:
# 모델 학습 함수 
def training(X_train, y_train, valTS, valTagetTS):   
    #[[],[]] <= [train, val] 
    loss_history=[[],[]]
    r2_history=[[],[]]
    
    for epoch in range(EPOCH):
        # 배치 손실 저장 변수
        bs_loss,bs_r2=0,0
        
        # 배치크기 만큼 학습 진행
        for i in range(BATCH_CNT):
            start = i*BATCH_SIZE 
            end = start + BATCH_SIZE 
            
            #- (0) BS크기만큼만 데이터 추출해서 Tensor화 진행
            BSX_train = torch.FloatTensor(X_train[start:end].values).to(DEVICE)
            BSy_train = torch.FloatTensor(y_train[start:end].values).to(DEVICE)
            # print(BSX_train.shape, BSX_train.device, BSX_train.dtype)
            # print(BSy_train.shape, BSy_train.device, BSy_train.dtype)
            
            #-(1)학습진행 - forward
            pre_y=model(BSX_train)
            #print(f'pre_y.shape : {pre_y.shape}')

            #-(2)오차계산 - 손실함수
            loss=F.mse_loss(pre_y, BSy_train)
            bs_loss += loss.item()
            bs_r2 += R2Score()(pre_y,BSy_train).item()

            #-(3)최적화 - 가중치,절편 업데이트 backward
            adam_optim.zero_grad()
            loss.backward()
            adam_optim.step()

        #-(4)검증 - 모델이 제대로 만들어지는 검사용
        val_loss, val_r2=testing(valTS, valTagetTS)
        loss_history[1].append(val_loss.item())
        r2_history[1].append(val_r2.item())
        
        # 에포크 단위 손실과 성능지표
        loss_history[0].append(bs_loss/BATCH_CNT)
        r2_history[0].append(bs_r2/BATCH_CNT)

        #-(4)학습결과 출력 및 저장
        print(f'[{epoch}/{EPOCH}]\n-TRAIN LOSS : {loss_history[0][-1]} R2 : {r2_history[0][-1]}')
        print(f'-VALID LOSS : {loss_history[1][-1]} R2 : {r2_history[1][-1]}')
        
    return loss_history, r2_history

In [21]:
# 모델 학습 진행
loss, r2 =training(X_train, y_train , X_val, y_val)

torch.Size([24, 1])
[Val] LOSS : 0.08007702976465225, R2 : 0.8081269860267639
[0/10000]
-TRAIN LOSS : 0.7268634187057614 R2 : -0.21756529808044434
-VALID LOSS : 0.08007702976465225 R2 : 0.8081269860267639
torch.Size([24, 1])
[Val] LOSS : 0.1365332454442978, R2 : 0.6728519201278687
[1/10000]
-TRAIN LOSS : 0.18484342051669955 R2 : 0.6982684284448624
-VALID LOSS : 0.1365332454442978 R2 : 0.6728519201278687
torch.Size([24, 1])
[Val] LOSS : 0.055831488221883774, R2 : 0.8662218451499939
[2/10000]
-TRAIN LOSS : 0.10471383854746819 R2 : 0.8224838227033615
-VALID LOSS : 0.055831488221883774 R2 : 0.8662218451499939
torch.Size([24, 1])
[Val] LOSS : 0.043744172900915146, R2 : 0.895184338092804
[3/10000]
-TRAIN LOSS : 0.07113391859456897 R2 : 0.8786309063434601
-VALID LOSS : 0.043744172900915146 R2 : 0.895184338092804
torch.Size([24, 1])
[Val] LOSS : 0.058714646846055984, R2 : 0.8593134880065918
[4/10000]
-TRAIN LOSS : 0.06645275093615055 R2 : 0.8857069462537766
-VALID LOSS : 0.058714646846055984 R

In [ ]:
# 학습 후 loss & r2 시각화
import matplotlib.pyplot as plt

THRESHOLD=EPOCH
fg, axes=plt.subplots(1,2, figsize=(10,5), sharex=True)
axes[0].plot(range(1, THRESHOLD+1), loss[0][:THRESHOLD], label='Train')
axes[0].plot(range(1, THRESHOLD+1), loss[1][:THRESHOLD], label='Val')
axes[0].grid()
axes[0].legend()
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("LOSS")

axes[1].plot(range(1, THRESHOLD+1), r2[0][:THRESHOLD], label='Train')
axes[1].plot(range(1, THRESHOLD+1), r2[1][:THRESHOLD], label='Val')
axes[1].grid()
axes[1].legend()
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("R2")
axes[1].set_title("R2")
plt.tight_layout()
plt.show()